In [1]:
import pandas as pd
import numpy as np
import os
os.listdir('../input')

['df_ref2_session.csv',
 'new_submission_dwell_time.csv',
 'new_submission_baseline.csv',
 'df_ref.csv',
 'df_dwell_time_session.csv',
 'ref_count2_baselineOnly_pred.csv',
 'train.csv',
 'df_ref2.csv',
 'submission_popular.csv',
 'item_metadata.csv',
 'df_full_except_noinfo.csv',
 'test.csv',
 'new_submission.csv',
 'dwell_df.csv',
 'baseline_sorting.csv',
 'dwell_time_agg.csv',
 'test2.csv']

In [2]:
path = '../input/'

In [15]:
ref_count_df = pd.read_csv(path + 'df_ref2_session.csv')
dwell_time_df = pd.read_csv(path + 'df_dwell_time_session.csv')

print('ref_count shape : ', ref_count_df.shape)
print('dwell_time shape : ', dwell_time_df.shape)

baseline shape :  (121526, 6)
ref_count shape :  (2322841, 4)
dwell_time shape :  (1943901, 3)


# merge

In [16]:
ref_count_df.ref_count = ref_count_df.ref_count.apply(int)
ref_count_df.head(3)

,user_id,session_id,reference,ref_count
0,0001VQMGUI65,466687f1875ee,3133074,11
1,0001VQMGUI65,466687f1875ee,2019467,1
2,0001VQMGUI65,466687f1875ee,4521140,1


In [5]:
dwell_time_df.head(3)

,session_id,reference,dwell_time
0,000056cd97ce2,3811810,23559
1,000056cd97ce2,2784325,28
2,00008c1f39c46,3501822,41422


In [17]:
merged = pd.merge(ref_count_df, dwell_time_df)
del ref_count_df, dwell_time_df
merged.shape

(1943901, 5)

In [20]:
merged['dwell_min'] = merged.dwell_time / 60

merged['ref-min'] = merged.ref_count / merged.dwell_min
merged['min-ref'] = merged.dwell_min / merged.ref_count

In [33]:
merged.rename(columns = {'reference' : 'iid'}, inplace = True)
merged.head(3)

,user_id,session_id,iid,ref_count,dwell_time,dwell_min,ref-min,min-ref
0,0001VQMGUI65,466687f1875ee,3133074,11,386,6.433333,1.709845,0.584848
1,0001VQMGUI65,466687f1875ee,2019467,1,69,1.150000,0.869565,1.150000
2,0001VQMGUI65,466687f1875ee,4521140,1,15588,259.800000,0.003849,259.800000


In [28]:
baseline = pd.read_csv(path + 'new_submission.csv')
baseline.impressions = baseline.impressions.apply(lambda x : x.split('|'))
print('baseline shape : ', baseline.shape)
baseline.head(3)

baseline shape :  (121526, 6)


,user_id,session_id,timestamp,step,reference,impressions
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 2033381, 1724779, 127131, 399441, 10..."
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ..."
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[3565720, 2947584, 4115018, 2039671, 3836538, ..."


# ref-min

In [40]:
ref_min = merged[['session_id','iid','ref-min']]
ref_min = ref_min.groupby('session_id').apply(lambda x : x.sort_values(['ref-min'], ascending = False)).reset_index(drop = True)
ref_min.iid = ref_min.iid.apply(str)
ref_min = ref_min.groupby('session_id')['iid'].apply(lambda x : ' '.join(x)).to_frame().reset_index()
ref_min.iid = ref_min.iid.apply(lambda x : x.split(' '))
ref_min.head(3)

,session_id,iid
0,000056cd97ce2,"[2784325, 3811810]"
1,00008c1f39c46,[3501822]
2,0000be39860d7,"[445081, 79237, 76308, 4920008]"


In [41]:
ref_min = pd.merge(baseline, ref_min, how = 'left')
print(ref_min.shape)
ref_min.head(3)

(121526, 7)


,user_id,session_id,timestamp,step,reference,impressions,iid
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 2033381, 1724779, 127131, 399441, 10...","[1050068, 2059240, 3241426]"
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[3565720, 2947584, 4115018, 2039671, 3836538, ...",[2817590]


In [42]:
idx = ref_min.iid.isnull()
ref_min.loc[idx, 'iid'] = ref_min.loc[idx, 'impressions']

In [43]:
ref_min.head(3)

,user_id,session_id,timestamp,step,reference,impressions,iid
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 2033381, 1724779, 127131, 399441, 10...","[1050068, 2059240, 3241426]"
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...","[2714480, 4476010, 3843244, 3833012, 9017890, ..."
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[3565720, 2947584, 4115018, 2039671, 3836538, ...",[2817590]


In [37]:
def mrr(answer, imp) :
    myscore = 0
    
    for i,j in zip(answer, imp) :
        if i in j :
            myscore += 1/(j.index(i) + 1)
    print(myscore / len(imp))

In [44]:
ref_min.reference = ref_min.reference.apply(str)
mrr(ref_min['reference'], ref_min['iid'])

0.5216253624927869


# min-ref

In [45]:
min_ref = merged[['session_id','iid','min-ref']]
min_ref = min_ref.groupby('session_id').apply(lambda x : x.sort_values(['min-ref'], ascending = False)).reset_index(drop = True)
min_ref.iid = min_ref.iid.apply(str)
min_ref = min_ref.groupby('session_id')['iid'].apply(lambda x : ' '.join(x)).to_frame().reset_index()
min_ref.iid = min_ref.iid.apply(lambda x : x.split(' '))
min_ref.head(3)

,session_id,iid
0,000056cd97ce2,"[3811810, 2784325]"
1,00008c1f39c46,[3501822]
2,0000be39860d7,"[4920008, 76308, 79237, 445081]"


In [46]:
min_ref = pd.merge(baseline, min_ref, how = 'left')
print(min_ref.shape)
min_ref.head(3)

(121526, 7)


,user_id,session_id,timestamp,step,reference,impressions,iid
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 2033381, 1724779, 127131, 399441, 10...","[3241426, 2059240, 1050068]"
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[3565720, 2947584, 4115018, 2039671, 3836538, ...",[2817590]


In [47]:
idx = min_ref.iid.isnull()
min_ref.loc[idx, 'iid'] = min_ref.loc[idx, 'impressions']

In [48]:
min_ref.reference = min_ref.reference.apply(str)
mrr(min_ref['reference'], min_ref['iid'])

0.6389217441361192


# ref * dwell

In [51]:
merged['refXdwell'] = merged['ref_count'] * merged['dwell_min']
merged.head(3)

,user_id,session_id,iid,ref_count,dwell_time,dwell_min,ref-min,min-ref,refXdwell
0,0001VQMGUI65,466687f1875ee,3133074,11,386,6.433333,1.709845,0.584848,70.766667
1,0001VQMGUI65,466687f1875ee,2019467,1,69,1.150000,0.869565,1.150000,1.150000
2,0001VQMGUI65,466687f1875ee,4521140,1,15588,259.800000,0.003849,259.800000,259.800000


In [53]:
refXdwell = merged[['session_id','iid','refXdwell']]
refXdwell = refXdwell.groupby('session_id').apply(lambda x : x.sort_values(['refXdwell'], ascending = False)).reset_index(drop = True)
refXdwell.iid = refXdwell.iid.apply(str)
refXdwell = refXdwell.groupby('session_id')['iid'].apply(lambda x : ' '.join(x)).to_frame().reset_index()
refXdwell.iid = refXdwell.iid.apply(lambda x : x.split(' '))
refXdwell.head(3)

,session_id,iid
0,000056cd97ce2,"[3811810, 2784325]"
1,00008c1f39c46,[3501822]
2,0000be39860d7,"[76308, 79237, 445081, 4920008]"


In [54]:
refXdwell = pd.merge(baseline, refXdwell, how = 'left')
print(refXdwell.shape)
idx = refXdwell.iid.isnull()
refXdwell.loc[idx, 'iid'] = refXdwell.loc[idx, 'impressions']

(121526, 7)


In [55]:
refXdwell.reference = refXdwell.reference.apply(str)
mrr(refXdwell['reference'], refXdwell['iid'])

0.6332330727165942
